In [16]:
data = open('C:\\Users\\v.todosiuk\\Desktop\\ML\\ML\\kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print ('data has %d chars, %d unique' % (data_size, vocab_size))

data has 276386 chars, 83 unique


Encode/Decode char/vector
Neural networks operate on vectors (a vector is an array of float) So we need a way to encode and decode a char as a vector.

We'll count the number of unique chars (vocab_size). That will be the size of the vector. The vector contains only zero exept for the position of the char wherae the value is 1.

So First let's calculate the vocab_size:

In [17]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print (char_to_ix)
print (ix_to_char)

{'A': 0, ':': 1, 'z': 2, 'Y': 3, ',': 4, 'F': 5, '\n': 6, '5': 7, '8': 8, 'J': 9, ';': 10, 'T': 11, 't': 12, 'G': 13, 'd': 14, 'K': 15, 'B': 16, 'Q': 17, '2': 18, 'I': 19, 'b': 20, 'я': 21, 'v': 22, 'N': 23, 'u': 24, 'a': 25, 'l': 26, 'W': 27, ' ': 28, 'L': 29, 'p': 30, 'D': 31, 'V': 32, 'k': 33, '6': 34, 'c': 35, '1': 36, '"': 37, 'o': 38, 'M': 39, "'": 40, 'x': 41, '-': 42, '@': 43, 'ю': 44, '(': 45, 'з': 46, ')': 47, 'P': 48, '\x00': 49, 'X': 50, 'U': 51, 'H': 52, 'i': 53, '*': 54, 'n': 55, 'y': 56, '/': 57, 'C': 58, 'w': 59, 'O': 60, '9': 61, 'e': 62, '.': 63, 'f': 64, '!': 65, 'j': 66, 'E': 67, 'g': 68, 'r': 69, '3': 70, '%': 71, '?': 72, '4': 73, 'q': 74, '0': 75, '7': 76, 's': 77, 'm': 78, '$': 79, 'h': 80, 'S': 81, 'R': 82}
{0: 'A', 1: ':', 2: 'z', 3: 'Y', 4: ',', 5: 'F', 6: '\n', 7: '5', 8: '8', 9: 'J', 10: ';', 11: 'T', 12: 't', 13: 'G', 14: 'd', 15: 'K', 16: 'B', 17: 'Q', 18: '2', 19: 'I', 20: 'b', 21: 'я', 22: 'v', 23: 'N', 24: 'u', 25: 'a', 26: 'l', 27: 'W', 28: ' ', 29: '

Finaly we create a vector from a char like this:
The dictionary defined above allosw us to create a vector of size 61 instead of 256.
Here and exemple of the char 'a'
The vector contains only zeros, except at position char_to_ix['a'] where we put a 1.

In [18]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print (vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Definition of the network
The neural network is made of 3 layers:

an input layer
an hidden layer
an output layer
All layers are fully connected to the next one: each node of a layer are conected to all nodes of the next layer. The hidden layer is connected to the output and to itself: the values from an iteration are used for the next one.

To centralise values that matter for the training (hyper parameters) we also define the sequence lenght and the learning rate

In [19]:
#model parameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [24]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [25]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 xEdJCrfXa7Ky)1g$з%'
54G:)6 OJJMfd'4xSяhge(hJw4agzBilM(bMMvT8Q$8PlN0ps8Xy2)J?
n;KY78L!з4VюH@! b3I/mb.e
Y,sbjH.6IYP?
gyXzHxJ4Eb3-%8*AvG
iiYr*8я5w2
f)k:p)w.oy!FfCTk
@fяy321zrlNO3'ollhk3G$2wFWh.YRxB'ooyUc 
----


In [26]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [21, 44, 60, 49, 55, 49, 62, 49, 28, 49, 78, 49, 38, 49, 69, 49, 55, 49, 53, 49, 55, 49, 68, 49, 4]
targets [44, 60, 49, 55, 49, 62, 49, 28, 49, 78, 49, 38, 49, 69, 49, 55, 49, 53, 49, 55, 49, 68, 49, 4, 49]


In [27]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print( 'iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter

iter 0, loss: 110.471020
----
 T,F';EE8i
BX?"fbBOW)cVXpa"!Bm5mHf%W!jRp
oJ*Sx'1C/()Xю@imNJ0юnF7@rю70zs 21@R-Hю%wwю?KJjA3(tk2wWE9jfD/S5Vw;,9w@IgX8hяR-з@2U
bYQ5cvP11(Y$4lcю!
3QirxkEq@юeз5 bqb4-V:SkJy5;!3t,V!ILTLTD54@JdN"od1,!N7(7% !XS 
----
iter 1000, loss: 66.098762
----
 d w d p e e     u n u u l   l s e r h p t '   l t t l e d l f i , l l   p f l s l l   o r l   l r   o s l a t   w l l h h l t o c y b a r e S     e h   t l c p e     d l m l   . b o l l     e     v l  
----
iter 2000, loss: 48.687501
----
 i k k b h   o f e i o   s o n o w r s n d .   r a d   t t e s e   i h e o   b m , e p g   s m   n e i n         f h r r n   r W p a t ; a u e o t o e   k   o   e r e t o   , e g s o h d t     n e p a  
----
iter 3000, loss: 41.186724
----
 w   a a a , a t e o g   t o o   n e   r r i u e e c i k h t o u p o d h s   n e o i o t u n r h e   r g g   y   l   o n n g   l e r e e e o s i r e e e a a l a n     a e e   x   w e b h e v g i a i t  
----
iter 4000, loss: 38.216379
----
 u s t r  

iter 35000, loss: 26.672189
----
  r o t ' s   w h a l l   a n d   a n d   x l a l i n g   t h e   s h o r g   r e   m o u l d   w o n i i t h o f s l e c h e   f r o n s s o   m o r s   i c h i n g   h e o n ' l ,   t h e   h e   h e 
----
iter 36000, loss: 25.615349
----
    t h e   h e   s f o a d   o f   t h e   t h e   f o r ;   n e k   t h e   t h e   H a m   s r o u t   k a i n ,   b u r   b a r s t c l i s   h i g h   t o   h i c r e l d   p w o f   w h e r   t i 
----
iter 37000, loss: 24.783262
----
  p o f   f r e t i n l e r   s r o d   r e e g o u s t   t h t h e d   w a m   s e r e r i n d   t o   i t l y   w a t   d i w   a   d e o t   s o o l d   w h .   
 n e d   n o t h i n d   h e   c o r 
----
iter 38000, loss: 24.290603
----
  e   o n s e   o v e n s s ,   b e a g a a n g o r   a p t   u n   n h i m e ,   e v e n i g h i s t e   e l e l e   h a v e   e v o u d t s   s i s   s o e d   k e a r d   w i k n   t i n g   t o   o 
----
iter 39000, loss: 24.251599
----
   

iter 69000, loss: 23.894392
----
 t ;   t h i s   h a d   h a d e r ,   t u o r p e l d   m o n c o   w a s   s e m e s   s u m e n e   p o   d u e g h e a d   d o s m / u r   t h e   w o u g h   i n c t e d i t n ,   h e s e   o f    
----
iter 70000, loss: 23.074621
----
 r ,   a r p a l l i n g l e t k e l y , .   G l h   s k a n d   a l d e d   w h e l l   c l o s   h e   w a t   b a y s e   t h a s   a l y i n g ,   h i r g   m o r e r e   f u r ,   t h e r   t h e  
----
iter 71000, loss: 22.638947
----
     w h e e   t h e   c o u l d   n e p r a r n .   G u t h   w h i t h l e   m o u c h   f o n   t h e   w h e y   s h o u t   h e   t o   m a t h   o u t   k h i s i n   b u t   t o   m a s e g o r  
----
iter 72000, loss: 22.497155
----
   b e   c o   s h e n t   s t i n d   w a s   u p   b e i n g   r e m e d   i n   t h e   h a a   s o n s   t h e   c o u u s h e m   G r e g u r e r   h o r   i l l   h e   d o o u d   o v e n s   e  
----
iter 73000, loss: 22.719864
----
 r 